# Selecting Nerual Network's Hyperparameters:
 I have considered here multiple combinations of:
   * **Number of Hidden Layers** 
   * **Number of Neurons** 
   * **Learning Rate:** is a constant that determine how rapidly we progress toward the minimum of our cost function.
   * **Number of Epoc:**  number of times an ENTIRE dataset is passed forward and backward through the neural network.  
   * **Error Margin:**  is the margin between cost function between current and previous modules during gradient descent.
   * **Regularization Parameter:** is a constant that reduces the values of particular weights. This is introduced in order to reduce the problem of overfitting 
   * **Number Fold** is the numbers of foldes that training data are split into. This is used in cross valdiation excercise. 

I plug a serie of combinations in `create_classifier` in order to measure the weighted overall accuracy on unseen data. I, then, select the hyperparameters that got the best accuracy to go into the final module that I used for prediction on test data.  
 

In [1]:
# Neural network|selecting_hyperparameters

import numpy as np
import itertools
import time

start_time = time.process_time()

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(int)
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
test_data = testing_spam[:, 1:]
test_labels = testing_spam[:, 0]

class hidden_layer:
    def __init__(self,n_inputs, n_of_neurons):
        self.weights = np.random.rand(n_inputs, n_of_neurons)
        self.biases = np.zeros([n_of_neurons,1])

    def feed_forward(self, inputs):
        return np.dot(self.weights.T, inputs) + self.biases

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

class neural_network:
    def __init__(self, number_of_hidden_layers=2, number_of_neurons=[9,9], learning_rate=1.8, max_n_epoc=1000, error_margin= 10**-9, regularization_parameter = 0.8):
        self.number_of_hidden_layers= number_of_hidden_layers
        self.number_of_neurons = number_of_neurons
        self.learning_rate = learning_rate
        self.max_n_epoc = max_n_epoc
        self.error_margin = error_margin
        self.regularization_parameter = regularization_parameter
        self.layer = []
        self.intera = []
        self.output = []
        self.j = 0
        self.Back_prop_residual = [0] * (self.number_of_hidden_layers + 1)

    def FeedForward(self, x):
        if self.j == 0:
            first_hidden_layer = hidden_layer(x.shape[0], self.number_of_neurons[0])
            intera = first_hidden_layer.feed_forward(x)
            output_1 = first_hidden_layer.sigmoid(intera)
            self.layer.append(first_hidden_layer)
            self.intera.append(intera)
            self.output.append(output_1)
            output_x = output_1
            for i in range(1, self.number_of_hidden_layers):
                hidden_layer_x = hidden_layer(self.number_of_neurons[i - 1], self.number_of_neurons[i])
                intera = hidden_layer_x.feed_forward(output_x)
                output_x = hidden_layer_x.sigmoid(intera)
                self.layer.append(hidden_layer_x)
                self.intera.append(intera)
                self.output.append(output_x)
            output_layer = hidden_layer(self.number_of_neurons[-1], 1)
            intera = output_layer.feed_forward(output_x)
            output_final = output_layer.sigmoid(intera)
            self.layer.append(output_layer)
            self.intera.append(intera)
            self.output.append(output_final)
        else:
            intera = self.layer[0].feed_forward(x)
            self.intera[0] = intera
            output_1 = self.layer[0].sigmoid(intera)
            self.output[0] = output_1
            output_x = output_1
            for i in range(1, self.number_of_hidden_layers):
                intera = self.layer[i].feed_forward(output_x)
                self.intera[i] = intera
                output_x = self.layer[i].sigmoid(intera)
                self.output[i] = output_x
            intera = self.layer[-1].feed_forward(output_x)
            self.intera[-1] = intera
            output_final = self.layer[-1].sigmoid(intera)
            self.output[-1] = output_final
        return self.output[-1]

    def BackProp(self, X, y_true):
        Back_prop = [0] * len(self.layer)
        Back_prop[-1] = self.output[-1] - y_true
        for i in range(1, len(self.layer)):
            Back_prop[-i-1] = np.multiply(np.dot(self.layer[-i].weights, Back_prop[-i]) , self.derv_Sigmoid(self.output[-i-1]))
            
        for i in range(len(self.layer)):
            if i==0 :
                self.Back_prop_residual[i] = self.Back_prop_residual[i] + np.dot(Back_prop[0], X)

            else:
                self.Back_prop_residual[i] = self.Back_prop_residual[i] + np.dot(Back_prop[i],self.output[i-1].T)

            
        for i in range(len(self.layer)):
            
            self.layer[i].weights -= self.learning_rate * (
                    self.Back_prop_residual[i].T / X.shape[
                0] + self.regularization_parameter * self.layer[i].weights)
            
            self.layer[i].biases -= self.learning_rate * (
                        np.sum(Back_prop[i], axis=1).reshape(self.layer[i].biases.shape) / X.shape[0])

    def train(self, X, y):
        n_epoc= 0
        temp_cost = 0

        while n_epoc <  self.max_n_epoc:
            y_predicted_temp = self.FeedForward(X.T).T
            current_cost = self.CostFunction(y, y_predicted_temp)
            self.j += 1
            delta = current_cost - temp_cost
            if abs(delta) > self.error_margin:
                temp_cost = current_cost
                self.BackProp(X, y)
            else:
                return n_epoc
            n_epoc += 1
        return n_epoc


    def CostFunction(self, y, y_predicted):
        weights_summed = 0
        for i in range(len(self.layer)):
            weights_summed += np.sum(np.sum(self.layer[i].weights**2, axis=0))
        return -(1/y.shape[0])* np.sum((y * np.log(y_predicted) + (1-y) * np.log(1-y_predicted))) + (self.regularization_parameter/ (2*y.shape[0]) * weights_summed)

    def derv_Sigmoid(self, x):
        return np.multiply(x , (1 - x))

    def predict(self, new_data):
        output_final = self.FeedForward(new_data)
        return np.where(output_final<=0.5,0,1)


def cross_validation(training_spam, item):
    number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter, k_fold = item
    x_training = training_spam[:, 1:]
    y_training = training_spam[:, 0]
    x = np.split(x_training, x_training.shape[0] / k_fold, axis=0)
    y = np.split(y_training, y_training.shape[0] / k_fold, axis=0)
    max_accuracy = 0
    averaged_accuracy = 0
    for i in range(len(x)):
        x_temp = np.concatenate(x[0:i] + x[i + 1:])
        y_temp = np.concatenate(y[0:i] + y[i + 1:])
        classifier = neural_network(number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter)
        n_epoc = classifier.train(x_temp, y_temp)

        final_y_predict_temp = classifier.predict(x[i].T).T
        accuracy = np.count_nonzero(final_y_predict_temp == y[i].reshape(final_y_predict_temp.shape)) / \
                   final_y_predict_temp.shape[0]

        if accuracy > max_accuracy:
            max_accuracy = accuracy
            max_classifier = classifier
        averaged_accuracy += accuracy
    averaged_accuracy = averaged_accuracy / len(x)
    return max_classifier, max_accuracy, averaged_accuracy, n_epoc


def create_classifier(training_spam, item):
    classifier = cross_validation(training_spam, item)
    return classifier


number_of_hidden_layers = [3]
number_of_neurons=[[3,4,2]]
learning_rate = [0.001,0.01, 0.1, 1, 2]
max_n_epoc = [10000, 15000]
error_margin= [10**-6, 10**-7]
regularization_parameter = [1,2,3]
k_fold =[200]



output_list = list(itertools.product(number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_parameter, k_fold))


dic = 0
max_weighted_accuracy=0
for item in output_list:
    classifier,max_accuracy_train, averaged_accuracy_train, n_epoc = create_classifier(training_spam, item)
    end_time = time.process_time()

    predictions = classifier.predict(test_data.T).T
    accuracy = np.count_nonzero(predictions == test_labels.reshape(test_labels.shape[0],1)) / test_labels.shape[0]
    weighted_accuracy = round(accuracy * test_data.shape[0] / (test_data.shape[0] + (training_spam.shape[0] / item[-2])) \
                              + averaged_accuracy_train * (training_spam.shape[0] / item[-2]) / (
                                          test_data.shape[0] + (training_spam.shape[0] / item[-2])), 3)

    if weighted_accuracy > max_weighted_accuracy:
        dic = {}

        max_weighted_accuracy = weighted_accuracy
        max_accuracy = accuracy
        max_averaged_accuracy_train = averaged_accuracy_train
        maxi_accuracy_train = max_accuracy_train

        max_item = item

        max_time = int(end_time - start_time)
    elif weighted_accuracy == max_weighted_accuracy:
        if accuracy > max_accuracy:
            max_weighted_accuracy = weighted_accuracy
            max_accuracy = accuracy
            maxi_accuracy_train = max_accuracy_train
            max_averaged_accuracy_train = averaged_accuracy_train
            max_item = item
            max_time = int(end_time - start_time)


    print("&&" * 50)
    print(
        f"Weighted accuracy is {weighted_accuracy} when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_folds are {item} respectively.")
    print()
    print(f"Accuracy on test data is: {accuracy}, averaged accuracy on train data is: {averaged_accuracy_train} after {n_epoc} epoc")
    print(f"This took", int(end_time - start_time), "seconds to solve.")
    print()
    print()
    print(
        f"**The best overall accuracy on test data is so far standing at: {max_weighted_accuracy} when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are {max_item}")
        print()
    print()
    print()

print()
print('^' * 250)
print(
    f"When number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are {max_item} respectively, the expected overall weighted accuracy on unseen data is: {max_weighted_accuracy}")
print(f"This module is {max_accuracy} accurate on test data")
print(f"Whereas, averaged accuracy on train data is {max_averaged_accuracy_train}, and topped at {maxi_accuracy_train}")
print(f"This module takes {max_time} seconds to solve.")

# pass on the result to the final module below
number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_parameter, k_fold = max_item





&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.841 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_folds are (3, [3, 4, 2], 0.001, 10000, 1e-06, 1, 200) respectively.

Accuracy on test data is: 0.918, averaged accuracy on train data is: 0.8019999999999999 after 10000 epoc
This took 583 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.841 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are (3, [3, 4, 2], 0.001, 10000, 1e-06, 1, 200)
**Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, averaged_accuracy_train ,max_accuracy_train, max_time]) : {}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.92

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.921 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_folds are (3, [3, 4, 2], 0.001, 15000, 1e-07, 2, 200) respectively.

Accuracy on test data is: 0.928, averaged accuracy on train data is: 0.913 after 15000 epoc
This took 8202 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.927 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are (3, [3, 4, 2], 0.001, 10000, 1e-06, 3, 200)
**Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, averaged_accuracy_train ,max_accuracy_train, max_time]) : {}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.902 when numbe

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.606 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_folds are (3, [3, 4, 2], 0.01, 15000, 1e-06, 3, 200) respectively.

Accuracy on test data is: 0.602, averaged accuracy on train data is: 0.613 after 332 epoc
This took 11156 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.927 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are (3, [3, 4, 2], 0.001, 10000, 1e-06, 3, 200)
**Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, averaged_accuracy_train ,max_accuracy_train, max_time]) : {}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.913 when number_

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.609 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_folds are (3, [3, 4, 2], 0.1, 15000, 1e-06, 1, 200) respectively.

Accuracy on test data is: 0.602, averaged accuracy on train data is: 0.613 after 106 epoc
This took 12410 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.927 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are (3, [3, 4, 2], 0.001, 10000, 1e-06, 3, 200)
**Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, averaged_accuracy_train ,max_accuracy_train, max_time]) : {}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.607 when number_o

<ipython-input-1-fdf23be20dc9>:123: RuntimeWarning: divide by zero encountered in log
  return -(1/y.shape[0])* np.sum((y * np.log(y_predicted) + (1-y) * np.log(1-y_predicted))) + (self.regularization_parameter/ (2*y.shape[0]) * weights_summed)
<ipython-input-1-fdf23be20dc9>:123: RuntimeWarning: invalid value encountered in multiply
  return -(1/y.shape[0])* np.sum((y * np.log(y_predicted) + (1-y) * np.log(1-y_predicted))) + (self.regularization_parameter/ (2*y.shape[0]) * weights_summed)


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.394 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_folds are (3, [3, 4, 2], 1, 10000, 1e-06, 3, 200) respectively.

Accuracy on test data is: 0.398, averaged accuracy on train data is: 0.38699999999999996 after 6 epoc
This took 13055 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.927 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are (3, [3, 4, 2], 0.001, 10000, 1e-06, 3, 200)
**Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, averaged_accuracy_train ,max_accuracy_train, max_time]) : {}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.609 whe

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.609 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_folds are (3, [3, 4, 2], 2, 10000, 1e-06, 1, 200) respectively.

Accuracy on test data is: 0.602, averaged accuracy on train data is: 0.613 after 10000 epoc
This took 16408 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.927 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are (3, [3, 4, 2], 0.001, 10000, 1e-06, 3, 200)
**Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, averaged_accuracy_train ,max_accuracy_train, max_time]) : {}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.392 when number_o

<ipython-input-1-fdf23be20dc9>:23: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.394 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_folds are (3, [3, 4, 2], 2, 10000, 1e-06, 3, 200) respectively.

Accuracy on test data is: 0.398, averaged accuracy on train data is: 0.38699999999999996 after 4 epoc
This took 16409 seconds to solve.


**The best overall accuracy on test data is so far standing at: 0.927 when number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter and k_fold are (3, [3, 4, 2], 0.001, 10000, 1e-06, 3, 200)
**Modules with similar accuracy (if any) have the following hyperparameters (hyperparameters:[weight accuracy, averaged_accuracy_train ,max_accuracy_train, max_time]) : {}



&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Weighted accuracy is 0.609 whe

#  Final Module 

When plugging the hyper parameters associated with the best average weighted accuracy I got when running the above code, I got a **0.93** accuracy when run on test data. 

In [26]:
class hidden_layer:
    def __init__(self,n_inputs, n_of_neurons):
        self.weights = np.random.rand(n_inputs, n_of_neurons)
        self.biases = np.zeros([n_of_neurons,1])

    def feed_forward(self, inputs):
        return np.dot(self.weights.T, inputs) + self.biases

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

class neural_network:
    def __init__(self, number_of_hidden_layers=3, number_of_neurons=[3,4,2], learning_rate=0.001, max_n_epoc=10000, error_margin= 10**-6, regularization_parameter = 3):
        self.number_of_hidden_layers= number_of_hidden_layers
        self.number_of_neurons = number_of_neurons
        self.learning_rate = learning_rate
        self.max_n_epoc = max_n_epoc
        self.error_margin = error_margin
        self.regularization_parameter = regularization_parameter
        self.layer = []
        self.intera = []
        self.output = []
        self.j = 0
        self.Back_prop_residual = [0] * (self.number_of_hidden_layers + 1)

    def FeedForward(self, x):
        if self.j == 0:
            first_hidden_layer = hidden_layer(x.shape[0], self.number_of_neurons[0])
            intera = first_hidden_layer.feed_forward(x)
            output_1 = first_hidden_layer.sigmoid(intera)
            self.layer.append(first_hidden_layer)
            self.intera.append(intera)
            self.output.append(output_1)
            output_x = output_1
            for i in range(1, self.number_of_hidden_layers):
                hidden_layer_x = hidden_layer(self.number_of_neurons[i - 1], self.number_of_neurons[i])
                intera = hidden_layer_x.feed_forward(output_x)
                output_x = hidden_layer_x.sigmoid(intera)
                self.layer.append(hidden_layer_x)
                self.intera.append(intera)
                self.output.append(output_x)
            output_layer = hidden_layer(self.number_of_neurons[-1], 1)
            intera = output_layer.feed_forward(output_x)
            output_final = output_layer.sigmoid(intera)
            self.layer.append(output_layer)
            self.intera.append(intera)
            self.output.append(output_final)
        else:
            intera = self.layer[0].feed_forward(x)
            self.intera[0] = intera
            output_1 = self.layer[0].sigmoid(intera)
            self.output[0] = output_1
            output_x = output_1
            for i in range(1, self.number_of_hidden_layers):
                intera = self.layer[i].feed_forward(output_x)
                self.intera[i] = intera
                output_x = self.layer[i].sigmoid(intera)
                self.output[i] = output_x
            intera = self.layer[-1].feed_forward(output_x)
            self.intera[-1] = intera
            output_final = self.layer[-1].sigmoid(intera)
            self.output[-1] = output_final
        return self.output[-1]

    def BackProp(self, X, y_true):
        Back_prop = [0] * len(self.layer)
        Back_prop[-1] = self.output[-1] - y_true
        for i in range(1, len(self.layer)):
            Back_prop[-i-1] = np.multiply(np.dot(self.layer[-i].weights, Back_prop[-i]) , self.derv_Sigmoid(self.output[-i-1]))
            
        for i in range(len(self.layer)):
            if i==0 :
                self.Back_prop_residual[i] = self.Back_prop_residual[i] + np.dot(Back_prop[0], X)

            else:
                self.Back_prop_residual[i] = self.Back_prop_residual[i] + np.dot(Back_prop[i],self.output[i-1].T)

            
        for i in range(len(self.layer)):
            
            self.layer[i].weights -= self.learning_rate * (
                    self.Back_prop_residual[i].T / X.shape[
                0] + self.regularization_parameter * self.layer[i].weights)
            
            self.layer[i].biases -= self.learning_rate * (
                        np.sum(Back_prop[i], axis=1).reshape(self.layer[i].biases.shape) / X.shape[0])

    def train(self, X, y):
        n_epoc= 0
        temp_cost = 0
        while n_epoc <  self.max_n_epoc:
            y_predicted_temp = self.FeedForward(X.T).T
            current_cost = self.CostFunction(y, y_predicted_temp)
            self.j += 1
            delta = current_cost - temp_cost
            if abs(delta) > self.error_margin:
                temp_cost = current_cost
                self.BackProp(X, y)
            else:
                return n_epoc
            n_epoc += 1
        return n_epoc


    def CostFunction(self, y, y_predicted):
        weights_summed = 0
        for i in range(len(self.layer)):
            weights_summed += np.sum(np.sum(self.layer[i].weights**2, axis=0))
        return -(1/y.shape[0])* np.sum((y * np.log(y_predicted) + (1-y) * np.log(1-y_predicted))) + (self.regularization_parameter/ (2*y.shape[0]) * weights_summed)

    def derv_Sigmoid(self, x):
        return np.multiply(x , (1 - x))

    def predict(self, new_data):
        new_data=new_data.T
        output_final = self.FeedForward(new_data)
        return np.where(output_final<=0.5,0,1).T
    

def cross_validation(training_spam):
    number_of_hidden_layers, number_of_neurons, learning_rate, max_n_epoc, error_margin, regularization_paramter, k_fold = item
    x_training = training_spam[:, 1:]
    y_training = training_spam[:, 0]
    x = np.split(x_training, x_training.shape[0] / k_fold, axis=0)
    y = np.split(y_training, y_training.shape[0] / k_fold, axis=0)
    max_accuracy = 0
    averaged_accuracy = 0
    for i in range(len(x)):
        x_temp = np.concatenate(x[0:i] + x[i + 1:])
        y_temp = np.concatenate(y[0:i] + y[i + 1:])
        classifier = neural_network()
        n_epoc = classifier.train(x_temp, y_temp)

        final_y_predict_temp = classifier.predict(x[i])
        accuracy = np.count_nonzero(final_y_predict_temp == y[i].reshape(final_y_predict_temp.shape)) / \
                   final_y_predict_temp.shape[0]

        if accuracy > max_accuracy:
            max_accuracy = accuracy
            max_classifier = classifier
        averaged_accuracy += accuracy
    averaged_accuracy = averaged_accuracy / len(x)
    return max_classifier


def create_classifier(training_spam):
    classifier = cross_validation(training_spam)
    return classifier

classifier = create_classifier(training_spam)



# import pickle
# a_file = open("data2.pkl", "wb")
# pickle.dump(classifier, a_file)
# a_file.close()

In [27]:
import numpy as np
import itertools
import time
import pickle

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(int)
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
test_data = testing_spam[:, 1:]
test_labels = testing_spam[:, 0]
a_file = open("data2.pkl", "rb")
output = pickle.load(a_file)
print(output)

In [28]:
# run on test data
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
test_data = testing_spam[:, 1:]
test_labels = testing_spam[:, 0]
test_labels = test_labels.reshape(test_labels.shape[0],1)

predictions = classifier.predict(test_data)
accuracy = np.count_nonzero(predictions == test_labels)/test_labels.shape[0]
print(f"Accuracy on test data is: {accuracy}")

Accuracy on test data is: 0.928


In [29]:
def confusion_matrix(test_labels, predictions):
    tp = tn = fp = fn = 0
    for actual_value, predicted_value in zip(test_labels, predictions):
        if predicted_value == actual_value:
            if predicted_value == 1: 
                tp += 1
            else: 
                tn += 1
        else: 
            if predicted_value == 1:
                fp += 1
            else: 
                fn += 1
    confusion_matrix = np.array([[tn, fp], [fn, tp]])
    print('___Confusion Matrix and Statistics___'.center(80))
    print('*****************************************'.center(80))
    print()
    print('           Reference   ')
    print ('Prediction   0    1')
    print('  0       ', confusion_matrix[0, 0],'  ',  confusion_matrix[0, 1])
    print('  1       ', confusion_matrix[1, 0], '  ', confusion_matrix[1, 1])
    
    
    accuracy = (tn+tp) / test_labels.shape[0]
    print()
    precision_positive =  tp / (tp+fp)
    precision_negative = tn / (tn+fn)
    recall_positive =  tp/ (tp+fn)
    recall_negative = tn/ (tn+fp)
    
    f1_positive = 2*(precision_positive * recall_positive)/(precision_positive+recall_positive)
    f1_negative = 2*(precision_negative * recall_negative)/(precision_negative+recall_negative)
    print()
    print()
    print('         Precision   Recall  f1-score   Support  ')
    print ('      0 ', np.round(precision_negative,2),'      ', np.round(recall_negative,2),'  ', np.round(f1_negative,2), '     ', tn+fp )
    print ('      1 ', np.round(precision_positive,2),'      ', np.round(recall_positive,2), '  ', np.round(f1_positive,2),'      ', tp+fn )
    print()
    print('accuracy                    ', accuracy,'    ',test_labels.shape[0] )

confusion_matrix(test_labels, predictions)



                     ___Confusion Matrix and Statistics___                      
                   *****************************************                    

           Reference   
Prediction   0    1
  0        283    18
  1        18    181



         Precision   Recall  f1-score   Support  
      0  0.94        0.94    0.94       301
      1  0.91        0.91    0.91        199

accuracy                     0.928      500
